# Cognition and Computation Project

## Import Libraries

In [1]:
import torch
import torch.nn as nn
import numpy as np

## Restricted Boltzmann Machine

In [2]:
class RBM(nn.Module):
    def __init__(self, n_visible, n_hidden, k=1):
        super(RBM, self).__init__()

        self.W = nn.Parameter(torch.randn(n_hidden, n_visible) * 0.01)
        self.h_bias = nn.Parameter(torch.zeros(n_hidden))
        self.v_bias = nn.Parameter(torch.zeros(n_visible))
        self.k = k

    def sample_h(self, v):
        prob_h = torch.sigmoid(torch.matmul(v, self.W.t()) + self.h_bias)
        return prob_h, torch.bernoulli(prob_h)

    def sample_v(self, h):
        prob_v = torch.sigmoid(torch.matmul(h, self.W) + self.v_bias)
        return prob_v, torch.bernoulli(prob_v)

    def forward(self, v):
        prob_h, h_sample = self.sample_h(v)
        for _ in range(self.k):
            prob_v, v_sample = self.sample_v(h_sample)
            prob_h, h_sample = self.sample_h(v_sample)
        return v_sample
    
    def train(self, dataset, learning_rate=0.1, epochs=5, batch_size=32, device='cpu'):
        self.to(device)
        dataset = dataset.to(device)

        # optimizer = torch.optim.SGD(self.parameters(), lr=learning_rate)
        # optimizer = torch.optim.Adagrad(self.parameters(), lr=learning_rate)
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)

        loss_history = []
        for epoch in range(epochs):
            epoch_loss_history = []
            for i in range(0, len(dataset), batch_size):
                v0 = dataset[i:i+batch_size]
                v = self.forward(v0)

                loss = torch.mean(torch.abs(v0-v))
                epoch_loss_history.append(loss.item())
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            epoch_loss = np.mean(epoch_loss_history)
            loss_history.append(epoch_loss)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss}")

        return loss_history


In [ ]:
device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)

print(f'Using device: {device_name}')

n_visible = 784  # e.g., for MNIST 28x28 images
n_hidden = 128
n_samples = 1000

# Generate random binary training data
train_data = torch.bernoulli(torch.rand(n_samples, n_visible))

# Initialize RBM
rbm = RBM(n_visible=n_visible, n_hidden=n_hidden, k=1)

# Train the RBM
loss_history = rbm.train(
    dataset=train_data,
    learning_rate=0.01,
    epochs=10,
    batch_size=32,
    device=device
)

## Deep Belief Network